In [1]:
#-*- coding:utf8 -*-
from PIL import Image
import pytesseract
#import tesseract
from pyocr import tesseract

im =Image.open('bb.png')

'''
#PIL中有九种不同模式。分别为1，L，P，RGB，RGBA，CMYK，YCbCr，I，F''
Convert()函数有三种形式的定义，它们定义形式如下：
im.convert(mode) ⇒ image
im.convert(“P”, **options) ⇒ image
im.convert(mode, matrix) ⇒ image
参考网址：http://blog.csdn.net/icamera0/article/details/50843172

'''
#(将图片转换为8位像素模式)
#im.convert("P")
#print(im.histogram())
print(im.mode)
##图片转换为灰色图像
im = im.convert('L')

pixdata = im.load()

rgb_im = im.convert('RGB')
r, g, b = rgb_im.getpixel((1, 1))
print(r, g, b)

'''
for y in range(im.size[1]):
    for x in range(im.size[0]):
        if pixdata[x,y][0]>170 and pixdata[x,y][1]>170 and pixdata[x,y][2]>170 and pixdata[x,y][3]>170:
            pixdata[x,y]=(255, 255, 255, 0)
        else:
            pixdata[x,y]=(0, 0, 0, 0)
        
im.save('a1.jpg')
'''
#im.show()
tesseract.image_to_string(im)
codes = tesseract.image_to_string(im)
print(pytesseract.image_to_string(im))
print(codes)

RGB
255 255 255
fig?
fig?


In [2]:
his = im.histogram()
values = {}
for i in range(256):
    values[i] = his[i]

for j,k in sorted(values.items(),key=lambda x:x[1],reverse=True)[:10]:
    print(j,k)

255 2615
254 149
62 121
253 82
252 57
250 45
248 40
251 36
249 32
237 28


In [3]:
im2 = Image.new("P",im.size,255)

for x in range(im.size[1]):
    for y in range(im.size[0]):
        pix = im.getpixel((y,x))
        if pix == 250 or pix == 252:
            im2.putpixel((y,x),0)
im2.show()

In [4]:
print(pytesseract.image_to_string(im2))

In [5]:
from PIL import Image,ImageDraw
from PIL import ImageEnhance
from PIL import ImageFilter
import sys
import re
from pytesseract import *


In [60]:
def getverify(name):
    #灰度转二值
    threshold  =  100
    table = []
    for i in range(256):
        if i< threshold:
            table.append(0)
        else:
            table.append(1)
    #矫正误识别为字母
    rep = { 'O':'0',
             'I':'1',
             'L':'1',
             'Z':'2',
             'S':'8',
             'Q':'0',
            '}':'7',
            '*':'',
            'E':'6',
            ']':'0',
            '`':'',
            'B':'8',
            '\\':'',
             ' ':'',
            'S':'5',
            '€':'8',
             'X':''
            }
    try:
        ims = Image.open(name)
        ims.load()
        imgry = ims.convert('L')
        imgry.save('g_'+name)
        out = imgry.point(table,'1')
        #out = imgry.point(lambda x: 0 if x < 1 else 255, '1')
        out.save('b_'+name)
        text = image_to_string(out,lang='eng',config="-psm 7").replace(' ','')
        text = re.sub("\W", "", text)
        text = text.strip()
        text = text.upper()
        for r in rep:
             text = text.replace(r,rep[r])
    except (Exception) as e:
        print(e)
    print(text)
    return text
getverify('p7.png')


17598


'17598'

In [57]:
print(pytesseract.image_to_string(Image.open('text3.png'),config="-psm 7"))

‘755198


In [59]:
def img_putpixel(filename):
    data = Image.open(filename)
    w,h = data.size
    #图像二值化
    enhancer = ImageEnhance.Contrast(data)
    data = enhancer.enhance(2)
    data = data.convert('1')
    #im.load()
    #data = im.getdata()
    print(w,h)
    try:
        for x in range(1,w-1):
            if x > 1 and x != w-2:
                #获取目标像素点左右位置
                left = x - 1
                right = x + 1
            for y in range(1,h-1):
                #获取目标像素点上下位置
                up = y - 1
                down = y + 1
                
                if x <= 2 or x >= (w - 2):
                    data.putpixel((x,y),255)
                    #print('image %s is processing...'%filename)
                elif y <= 2 or y >= (h - 2):
                    data.putpixel((x,y),255)
                    #print('image2 %s is processing...'%filename)
                elif data.getpixel((x,y)) == 0:
                    if y > 1 and y != h-1:
                        #以目标像素点为中心点，获取周围像素点颜色
                        #0为黑色，255为白色
                        up_color = data.getpixel((x,up))
                        down_color = data.getpixel((x,down))
                        left_color = data.getpixel((left,y))
                        left_down_color = data.getpixel((left,down))
                        right_color = data.getpixel((right,y))
                        right_up_color = data.getpixel((right,up))
                        right_down_color = data.getpixel((right,down))
                        #去除竖线干扰线
                        if down_color == 0:
                            if left_color == 255 and left_down_color == 255 and right_color == 255 and right_down_color == 255:
                                data.putpixel((x,y),255)
                                data.save("text2.png","png")
                                print('text2.png saved...')
                        #去除横线干扰线
                        elif right_color == 0:
                            if down_color == 255 and right_down_color == 255 and up_color == 255 and right_up_color == 255:
                                data.putpixel((x,y),255)
                                data.save("text3.png","png")
                                print('text3.png saved...')
                    #去除斜线干扰线
                    if left_color == 255 and right_color == 255 and up_color == 255 and down_color == 255:
                        data.putpixel((x,y),255)
                else:
                    pass
                #保存去除干扰线后的图片
                #data.load()
                data.save("test_02.png","png")
                #print('File is saved!')
    except (Exception) as e:
        print(e)
        return False
img_putpixel('g_p7.png')

120 40
text3.png saved...
text3.png saved...
text2.png saved...
text3.png saved...
text3.png saved...
text2.png saved...
text3.png saved...
text2.png saved...
text3.png saved...
text3.png saved...
text2.png saved...
text3.png saved...
text3.png saved...
text3.png saved...
text3.png saved...
text2.png saved...
text2.png saved...
text2.png saved...
text3.png saved...
text3.png saved...


In [53]:
def draw_img(name):
    #去除干扰线
    im = Image.open(name)
    #图像二值化
    enhancer = ImageEnhance.Contrast(im)
    im = enhancer.enhance(2)
    im = im.convert('1')
    #im.load()
    data = im.getdata()
    w,h = im.size
    #im.show()
    black_point = 0
    for x in range(1,w-1):
        for y in range(1,h-1):
            mid_pixel = data[w*y+x] #中央像素点像素值
            if mid_pixel == 0: #找出上下左右四个方向像素点像素值
                top_pixel = data[w*(y-1)+x]
                left_pixel = data[w*y+(x-1)]
                down_pixel = data[w*(y+1)+x]
                right_pixel = data[w*y+(x+1)]
                #判断上下左右的黑色像素点总个数
                if top_pixel == 0:
                    black_point += 1
                if left_pixel == 0:
                    black_point += 1
                if down_pixel == 0:
                    black_point += 1
                if right_pixel == 0:
                    black_point += 1
                    print(black_point)
                if black_point <= 3:
                    im.putpixel((x,y),255)
                    #print('black_point is %s:'%black_point)
        black_point = 0
    im.save('gg_'+name)
    text = image_to_string(im,lang='eng',config="-psm 7").replace(' ','')
    text = text.strip()
    text = text.upper()
    for r in rep:
        text = text.replace(r,rep[r])
    text = re.sub("\W", "", text)
    print(text)
    im.show()
draw_img('p4.png')
#print(a)

<class 'ImagingCore'>
2
4
6
8
11
13
14
3
6
10
13
16
20
27
30
33
4
8
12
16
19
22
26
29
3
6
10
19
23
27
30
2
4
7
15
19
23
26
2
4
8
16
20
23
2
4
8
13
17
20
2
5
12
16
19
2
4
7
10
14
17
2
4
12
16
19
2
3
4
6
9
14
18
21
2
4
7
11
15
2
5
8
10
14
2
5
8
11
14
2
4
8
12
15
19
2
5
8
11
15
19
23
2
4
8
12
16
18
22
28
2
5
9
14
17
20
24
27
2
5
8
12
14
17
21
25
2
5
8
11
13
15
19
23
29
3
6
10
13
16
20
24
27
31
35
38
41
43
2
5
9
13
17
21
25
29
33
37
41
44
4
8
12
16
20
33
37
40
8
22
24
6
8
7
9
11
14
17
20
7
10
14
18
22
2
5
9
13
17
20
3
5
8
12
25
27
2
3
5
8
12
20
2
3
5
9
13
3
5
9
2
5
8
2
4
7
8
11
14
17
19
1
3
5
9
21
25
2
4
7
10
2
4
7
11
15
18
2
5
8
10
13
17
21
3
6
10
13
15
22
2
5
8
11
14
16
2
5
9
11
13
17
2
4
8
11
14
17
21
2
5
9
13
17
21
5
9
12
14
18
4
8
12
15
18
22
25
26
28
3
6
10
14
18
25
27
30
2
5
9
13
19
22
3
6
10
15
18
2
5
8
11
14
5
3
2
3
2
3
5
7
2
4
7
9
11
17
2
4
8
12
15
2
4
8
12
16
20
22
2
4
8
12
16
20
24
27
3
5
8
12
25
29
32
2
4
7
9
12
16
24
28
30
2
4
7
11
15
19
21
23
27
35
38
2
5
9
13
17
21
25
28
32